# Synthesizing circuit diagrams from Q# code

In [ ]:
import qsharp

The `dump_circuit()` function displays a circuit that contains the gates that have been applied in the simulator up to this point.

In [ ]:
%%qsharp

// Prepare a Bell State.
use register = Qubit[2];
H(register[0]);
CNOT(register[0], register[1]);

In [ ]:
qsharp.dump_circuit()

If you have the Q# widgets installed, you can display the circuit as an SVG image.

_Run `pip install qsharp-widgets` in the command line to install the Q# widgets._

In [ ]:
from qsharp_widgets import Circuit

Circuit(qsharp.dump_circuit())

You can synthesize a circuit diagram for any program by calling `qsharp.circuit()` with an entry expression.

In [ ]:
%%qsharp

operation GHZSample(n: Int) : Result[] {
    use qs = Qubit[n];

    H(qs[0]);
    ApplyToEach(CNOT(qs[0], _), qs[1...]);

    let results = MeasureEachZ(qs);
    ResetAll(qs);
    return results;
}

In [ ]:
Circuit(qsharp.circuit("GHZSample(3)"))

Circuit diagrams can also be generated for any operation that takes qubits or arrays of qubits.

The diagram will show as many wires as there are input qubit, plus any additional qubits that are allocated within the operation.

When the operation takes an array of qubits (`Qubit[]`), the circuit will show the array as a register of 2 qubits.

In [ ]:
%%qsharp

operation PrepareCatState(register : Qubit[]) : Unit {
    H(register[0]);
    ApplyToEach(CNOT(register[0], _), register[1...]);
}

In [ ]:
Circuit(qsharp.circuit(operation="PrepareCatState"))

Circuit synthesis takes into account the currently chosen target, and will perform the same gate decompositions and other transformations that compiling for that target would produce.

Here, we show what the circuit looks like for a random bit generator when the Unrestricted target profile is chosen.

In [ ]:
%%qsharp

operation TwoRandomBits() : Result[] {
    let r1 = RandomBit();
    let r2 = RandomBit();
    return [r1, r2];
}

operation RandomBit() : Result {
    use q = Qubit();
    H(q);
    MResetZ(q)
}

In [ ]:
Circuit(qsharp.circuit("TwoRandomBits()"))

If we generate a circuit for the same program, but targeting the Base profile, the resulting circuit avoids reset gates and uses two qubits instead.

In [ ]:
qsharp.init(target_profile=qsharp.TargetProfile.Base)

In [ ]:
%%qsharp

operation TwoRandomBits() : Result[] {
    let r1 = RandomBit();
    let r2 = RandomBit();
    return [r1, r2];
}

operation RandomBit() : Result {
    use q = Qubit();
    H(q);
    MResetZ(q)
}

In [ ]:
Circuit(qsharp.circuit("TwoRandomBits()"))

Regardless of the target chosen, conditionals that compare `Result` values are not permitted during circuit synthesis. This is because they may introduce nondeterminism such that the circuit will look different depending on measurement outcome. Representing conditionals visually is not supported.

In [ ]:
qsharp.init(target_profile=qsharp.TargetProfile.Unrestricted)

In [ ]:
%%qsharp

operation ResetIfOne() : Result {
    use q = Qubit();
    H(q);
    let r = M(q);
    if (r == One) {
        Message("result was One, applying X gate");
        X(q);
    } else {
        Message("result was Zero");
    }
    Reset(q);
    return r
}

In [ ]:
# Program can be simulated. Differerent shots may produce different results.
print("Simulating program...")
qsharp.run("ResetIfOne()", 3)

print()

# The same program cannot be synthesized as a circuit because of the conditional X gate.
print("Synthesizing circuit for program (should raise error)...")
try:
    qsharp.circuit("ResetIfOne()")
except qsharp.QSharpError as e:
    print(e)

Even though we can't synthesize the above program into a circuit, we still have the option of running it in the simulator, and displaying the resulting circuit.

Note that the resulting circuit diagram shows only one of the two branches that could have been taken.

In [ ]:
%%qsharp

ResetIfOne()

In [ ]:
Circuit(qsharp.dump_circuit())